In [ ]:
import random
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc

# Configura el navegador con undetected_chromedriver
browser = uc.Chrome()

# URL inicial: Página de listados de Madrid
url = "https://www.pisos.com/venta/pisos-madrid/"
browser.get(url)

# Manejar las cookies
try:
    WebDriverWait(browser, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="didomi-notice-agree-button"]'))
    ).click()
except Exception as e:
    print("No se encontró el botón de cookies o ya fue aceptado.")

# Espera a que los inmuebles se carguen en la página principal
try:
    WebDriverWait(browser, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.ad-preview'))
    )
except Exception as e:
    print("TimeoutException: No se encontraron los inmuebles en el tiempo esperado.")
    browser.quit()

# Extraer el primer enlace de la lista de inmuebles
html = browser.page_source
soup = bs(html, 'lxml')

# Encuentra el primer contenedor de inmueble y obtiene el enlace
primer_inmueble = soup.find('div', {'class': 'ad-preview'})
primer_enlace = primer_inmueble.get('data-lnk-href')
primer_enlace_completo = f"https://www.pisos.com{primer_enlace}"

# Accede al primer enlace de inmueble
browser.get(primer_enlace_completo)
time.sleep(3)  # Espera a que la página del primer inmueble cargue

# Lista para almacenar los datos
all_data = []

# Bucle para iterar desde el primer inmueble a los siguientes inmuebles desde su página
while True:
    try:
        # Extraer la información del inmueble actual
        html_inmueble = browser.page_source
        soup_inmueble = bs(html_inmueble, 'lxml')

        # Extraer datos del inmueble
        try:
            # Precio
            precio = soup_inmueble.find('div', {'class': 'price__value jsPriceValue'}).text.split(' ')[0] if soup_inmueble.find('div', {'class': 'price__value jsPriceValue'}) else "N/A"

            # Superficie construida
            superficie_construida = soup_inmueble.find('span', {'class': 'features__value'}).text.split(' ')[0] if soup_inmueble.find('span', {'class': 'features__value'}) else "N/A"

            # Última actualización
            ultima_actualizacion = soup_inmueble.find('p', {'class': 'last-update__date'}).text if soup_inmueble.find('p', {'class': 'last-update__date'}) else "N/A"

            # Características
            c1 = soup_inmueble.find('div', {'class': 'features__content'})
            features_list = []
            if c1:
                features = c1.find_all('div', {'class': 'features__feature'})
                for feature in features:
                    label = feature.find('span', {'class': 'features__label'}).get_text(strip=True)
                    value = feature.find('span', {'class': 'features__value'}).get_text(strip=True)
                    features_list.append((label, value))

            # Certificado energético
            try:
                energy_certificate = soup_inmueble.find('div', {'class': 'details__block energy-certificate'})
                consumo = energy_certificate.find('div', {'class': 'energy-certificate__data'}).find_all('span')[1].get_text(strip=True)
                emisiones = energy_certificate.find_all('div', {'class': 'energy-certificate__data'})[1].find_all('span')[1].get_text(strip=True)
            except AttributeError:
                consumo = "N/A"
                emisiones = "N/A"

            # Guarda los datos en una lista
            data = {
                'Enlace': browser.current_url,
                'Precio': precio,
                'Superficie Construida': superficie_construida,
                'Última Actualización': ultima_actualizacion,
                'Consumo Energético': consumo,
                'Emisiones CO2': emisiones,
                'Características': features_list,
                'Tipo de operación': 'Compra'  # Se añade la operación como compra
            }

            all_data.append(data)

            # Imprime los datos de la página actual (opcional)
            print(f"Inmueble {len(all_data)}: {data}")

        except Exception as e:
            print(f"Error al extraer datos de {browser.current_url}: {e}")

        # Intenta hacer clic en el botón de "Siguiente" para ir al siguiente inmueble
        try:
            siguiente_boton = WebDriverWait(browser, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, ".navigation__link.navigation__link--next"))
            )
            siguiente_boton.click()

            # Pausa para esperar a que cargue la siguiente página
            time.sleep(random.uniform(3, 5))

        except Exception as e:
            # Si no se encuentra el botón "Siguiente", termina el bucle
            print("No se encontró el botón 'Siguiente' o ya no hay más inmuebles.")
            break

    except Exception as e:
        print(f"Error al cargar la página del inmueble: {e}")
        break

# Una vez terminado el bucle, puedes convertir los datos en un DataFrame de pandas y guardarlos
df = pd.DataFrame(all_data)
print(df)

# Opcional: guarda el DataFrame en un archivo CSV
df.to_csv('pisos_data_todos_inmuebles.csv', index=False)

# Cierra el navegador
browser.quit()